# Evaluation

In [1]:
import sys
import os
import importlib

# set your working dir
work_dir = '/home/yizi/IBL_foundation_model'
os.chdir(work_dir)
print('working dir: ', work_dir)

path = 'src'
sys.path.append(str(path))

from src.utils.eval_utils import load_model_data_local, co_smoothing_eval, compare_R2_scatter, behavior_decoding, draw_threshold_table

working dir:  /home/yizi/IBL_foundation_model


### 1.Configuration & Load
- The dataset will be split by `datasets.train_test_split(test_size=test_size, seed=seed)`

In [2]:
# Configuration
configs = {
    'model_config': 'src/configs/ndt1.yaml',
    'model_path': 'checkpoints/models/ndt1/ndt1_671c_unaligned_shuffle/model_best.pt',
    'trainer_config': 'src/configs/trainer.yaml',
    'dataset_path': 'data/671c7ea7-6726-4fbe-adeb-f89c2c8e489b_aligned', 
    'test_size': 0.15,
    'seed': 42,
}  

In [3]:
# load your model and dataloader
model, accelerator, dataset, dataloader = load_model_data_local(**configs)

seed set to 42
bin size: 0.02
len(dataset): 121
spike data shape: torch.Size([121, 100, 668])


### 2.Co-smoothing (From N-1 to 1) 
- using `co_smoothing_r2` for r2 and psth/residual plots.
- using `co_smoothing_bps` for co-bps stat.

In [ ]:
co_smoothing_configs = {
    'subtract': 'task',
    'onset_alignment': [40],
    'method_name': 'test', # used for file name of figures
    'save_path': 'figs/test',
    'mode': 'per_neuron',
    'n_time_steps': 100,    
    'is_aligned': True,
    'target_regions': None
}

co_smoothing_eval(model, accelerator, dataloader, dataset, **co_smoothing_configs)

### 3. Forward Prediction
- using `co_smoothing_r2` with mode `forward-pred` for r2 and psth/residual plots.
- using `co_smoothing_bps` with mode `forward-pred` for co-bps stat.

In [ ]:
co_smoothing_configs = {
    'subtract': 'task',
    'onset_alignment': [],
    'method_name': 'test', # used for file name of figures
    'save_path': 'figs/test',
    'mode': 'forward_pred',
    'n_time_steps': 100,    
    'held_out_list': list(range(80, 100)),
    'is_aligned': True,
    'target_regions': None
}

co_smoothing_eval(model, accelerator, dataloader, dataset, **co_smoothing_configs)

### 4. Inter-Region Prediction
- using `co_smoothing_r2` with mode `inter_region` for r2 and psth/residual plots.
- using `co_smoothing_bps` with mode `inter_region` for co-bps stat.

In [ ]:
co_smoothing_configs = {
    'subtract': 'task',
    'onset_alignment': [40],
    'method_name': 'test', # used for file name of figures
    'save_path': 'figs/test',
    'mode': 'inter_region',
    'n_time_steps': 100,    
    'held_out_list': None,
    'is_aligned': True,
    'target_regions': ['all']
}

co_smoothing_eval(model, accelerator, dataloader, dataset, **co_smoothing_configs)

### 5. Intra-Region Prediction
- using `co_smoothing_r2` with mode `intra_region` for r2 and psth/residual plots.
- using `co_smoothing_bps` with mode `intra_region` for co-bps stat.

In [ ]:
co_smoothing_configs = {
    'subtract': 'task',
    'onset_alignment': [40],
    'method_name': 'test', # used for file name of figures
    'save_path': 'figs/test',
    'mode': 'intra_region',
    'n_time_steps': 100,    
    'held_out_list': None,
    'is_aligned': True,
    'target_regions': ['all']
}

co_smoothing_eval(model, accelerator, dataloader, dataset, **co_smoothing_configs)

### 6. R2 Scatter Plot
- First, run 1 & 2 for each model you want to compare. A `r2.npy` file would be generated in the corresponding figs directory. (the same directory as the `save_path` in 2.)

In [ ]:
compare_r2_configs = {
    'A_path': 'figs/671c_ndt1_nm30_ualg_shuffle_best', # should be the same as the 'save_path' from above
    'B_path': 'figs/671c_ndt1_nm30_alg(-0.4)_shuffle_last',
    'A_name': 'unaligned',    # names for plots
    'B_name': 'aligned'
}    

compare_R2_scatter(**compare_r2_configs)

### 7. Behavior Decoding

In [2]:
# Configuration
configs = {
    'model_config': 'src/configs/ndt1.yaml',
    'model_path': '/home/yizi/IBL_foundation_model/results/method_ssl/mask_neuron/model_best.pt',
    'trainer_config': 'src/configs/trainer_sl_choice.yaml',
    'dataset_path': 'data/671c7ea7-6726-4fbe-adeb-f89c2c8e489b_aligned', 
    'save_path': 'figs/model_NDT1/method_ssl/mask_neuron/choice_decoding',
    'test_size': 0.2,
    'seed': 42,
    'mask_name': 'mask_neuron',
    'metric': 'acc',
    'from_scratch': False,
    'freeze_encoder': False
}  

In [ ]:
# load your model and dataloader
results = behavior_decoding(**configs)

### 8. Threshould Table
- TODO: quality threshold

In [ ]:
from ipywidgets import interact, widgets, fixed

mask_methods = ['']
eval_methods = ['co_smooth', 'forward_pred', 'intra_region', 'inter_region']

firing_rate_threshold = widgets.FloatRangeSlider(
    value=[0.5, 2.5],
    min=0.0,
    max=4.0,
    step=0.01,
    description='FR Ts:',
    orientation='horizontal',
    continuous_update=True
)

quality_threshold = widgets.FloatRangeSlider(
    value=[0.5, 2.5],
    min=0.0,
    max=5.0,
    step=0.1,
    description='Quality Ts:',
    orientation='horizontal',
    continuous_update=False
)

interact(
    draw_threshold_table,
    mask_methods=fixed(mask_methods),
    eval_methods=fixed(eval_methods),
    firing_rate_ts=firing_rate_threshold,
    load_path=fixed('figs'),
    quality_ts=quality_threshold
)